In [17]:
from og_marl.offline_dataset import calculate_returns
from flashbax.vault import Vault

In [20]:
vlt = Vault(vault_name="rware/tiny-2ag.vlt", vault_uid="Replay")

Loading vault found at /home/claude/Documents/og-marl/vaults/rware/tiny-2ag.vlt/Replay


In [21]:
exp = vlt.read(percentiles=(80,100))

In [23]:
rewards = exp.experience["reward"][0,:,0]
terminal = exp.experience["done"][0,:,0]

In [25]:
import numpy as np

ep_returns = []
ep_return = 0
for j in range(3):
    for i in range(len(terminal)):
        ep_return += rewards[i]

        if terminal[i] == True:
            ep_returns.append(ep_return)
            ep_return = 0


print(np.mean(ep_returns))

1.7028985


In [26]:
import jax.numpy as jnp
from flashbax.buffers.trajectory_buffer import TrajectoryBufferState

def transform_buffer_state(state):
    experience = {}
    experience["observations"] = state.experience["obs"]["agents_view"]
    experience["act"] = state.experience["action"]
    experience["rew"] = jnp.repeat(state.experience["reward"], axis=2, repeats=2)
    experience["done"] = jnp.repeat(state.experience["done"], axis=2, repeats=2)
    experience["legals"] = state.experience["obs"]["action_mask"]
    experience["mask"] = jnp.ones((*experience["act"].shape[:2],))
    experience["env_state"] = jnp.reshape(experience["obs"], (*experience["obs"].shape[:2],-1))
    state = TrajectoryBufferState(experience=experience, is_full=state.is_full, current_index=state.current_index)
    return state

In [27]:
new_state = transform_buffer_state(exp)

In [28]:
new_state.experience

{'obs': Array([[[[ 1,  0,  3, ...,  0,  1,  0],
          [ 0,  1,  1, ...,  0,  0,  0]],
 
         [[ 1,  0,  3, ...,  0,  1,  0],
          [ 0,  1,  1, ...,  0,  0,  0]],
 
         [[ 1,  0,  3, ...,  0,  1,  0],
          [ 0,  1,  1, ...,  0,  0,  0]],
 
         ...,
 
         [[ 1,  0,  5, ...,  0,  1,  0],
          [ 0,  1,  9, ...,  0,  0,  0]],
 
         [[ 1,  0,  5, ...,  0,  1,  0],
          [ 0,  1,  9, ...,  0,  0,  0]],
 
         [[ 1,  0,  5, ...,  0,  1,  0],
          [ 0,  1,  9, ...,  0,  0,  0]]],
 
 
        [[[ 1,  0,  1, ...,  0,  0,  0],
          [ 0,  1, 10, ...,  0,  0,  0]],
 
         [[ 1,  0,  1, ...,  0,  0,  0],
          [ 0,  1, 10, ...,  0,  0,  0]],
 
         [[ 1,  0,  1, ...,  0,  0,  0],
          [ 0,  1, 10, ...,  0,  0,  0]],
 
         ...,
 
         [[ 1,  0,  7, ...,  0,  1,  0],
          [ 0,  1,  6, ...,  0,  1,  0]],
 
         [[ 1,  0,  7, ...,  0,  1,  0],
          [ 0,  1,  6, ...,  0,  1,  0]],
 
         [[ 1,  0,  6, 